# Outline

This notebook include model training process and with optuna hyperparameter finding.

Preprocess and feature Engineering is base on previous notebook.

link : [https://www.kaggle.com/code/wesleyhuan/beginer-level-eda-season-6-episode-1](http://)

Include these steps:

* Load data
* Preprocess data
* Train model with Hyper parameter (OPTUNA)
* Submission

# Load data

In [1]:
# import necessary library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#for optuna pipline
import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


# Load data

In [2]:
# config
class CFG:
    train_csv = '/kaggle/input/playground-series-s6e1/train.csv'
    test_csv = '/kaggle/input/playground-series-s6e1/test.csv'
    sample_submission_csv = '/kaggle/input/playground-series-s6e1/sample_submission.csv'
    N_FOLDS = 5
    RANDOM_SEED = 42
    
#torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
train = pd.read_csv(CFG.train_csv)
test = pd.read_csv(CFG.test_csv)

In [4]:
test.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate


# Preprocess data

2026/01/13
change into class for pipline

In [5]:
class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.medians = {}
        self.encoders = {}
        self.numeric_cols = []
        self.all_non_numeric = []
        self.categorical_cols = []
        self.level_mapping = {'easy': 1, 'moderate': 2, 'hard': 3,
                              'low': 1, 'medium': 2, 'high': 3,
                              'poor': 1, 'average': 2, 'good': 3}
        self.level_cols = ['sleep_quality', 'facility_rating', 'exam_difficulty']
        
    def fit(self, df, y=None):
        """
        Learn the parameters (medians, categories) from the TRAINING data.
        """
        # Identify columns
        self.numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        self.all_non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
        self.categorical_cols = [c for c in self.all_non_numeric if c not in self.level_cols]
        
        # Learn Medians for numeric columns
        for col in self.numeric_cols:
            self.medians[col] = df[col].median()
            
        # Fit Encoders for categorical columns
        # handle_unknown='use_encoded_value' prevents crashes if Test data has new categories
        for col in self.categorical_cols:
            enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
            enc.fit(df[[col]].astype(str)) 
            self.encoders[col] = enc
            
        return self

    def transform(self, df):
        """
        Apply the learned parameters to the data (Train or Test).
        """
        df = df.copy()
        
        # Drop irrelevant columns (ID is usually dropped, Target handled separately)
        # Note: We don't drop target here to keep X and y aligned until the end
        if 'id' in df.columns:
            df = df.drop(columns=['id'])
        if 'exam_score' in df.columns:
            df = df.drop(columns=['exam_score'])

        # level encoding
        for col in self.level_cols:
            if col in df.columns:
                # ues mapping rule fillna with 0 
                df[col] = df[col].map(self.level_mapping).fillna(0).astype(int)
        # Impute Missing Values using LEARNED medians
        for col in self.numeric_cols:
            if col in df.columns:
                df[col] = df[col].fillna(self.medians.get(col, 0))
        
        # Apply Encoding
        for col in self.categorical_cols:
            if col in df.columns:
                # Fill NaN in categoricals with 'Missing' before encoding to be safe
                df[col] = df[col].astype(str).fillna('Missing')
                df[col] = self.encoders[col].transform(df[[col]]).flatten()

        # interaction features
        df = self.create_interaction_features(df)
        
        return df
        
    def create_interaction_features(self, df):# add because of the Correlation matrix
        df = df.copy()
        
        # sleep_hours * sleep_quality
        # assumption : sleep_quality might increase value of sleep_hours
        if 'sleep_hours' in df.columns and 'sleep_quality' in df.columns:
            df['sleep_hours_quality'] = df['sleep_hours'] * df['sleep_quality']
            
        # class_attendance * facility_rating
        # assumption : better facility_rating with increase the effect class_attendance
        if 'class_attendance' in df.columns and 'facility_rating' in df.columns:
            df['facility_rating_attendance'] = df['class_attendance'] * df['facility_rating']
            
        # study_method * study_hours
        # study_method might increase value of study_hours
        if 'study_method' in df.columns and 'study_hours' in df.columns:
            df['study_method_hours'] = df['study_method'] * df['study_hours']
        
        return df

In [6]:
X_train_raw = train#.drop(columns=['exam_score'])
y_train = train['exam_score']

# Train model with Hyper parameter (OPTUNA) 

2026/01/13 reach TOP 30%
1. add optuna for hyperparameter 
2. cross validation 
3. combine preprocessor and model into pip line

In [7]:
def objective(trial):

    model_params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
    }

    pipeline = Pipeline(
        steps=[
            ("preprocess", Preprocessor()),
            ("model", XGBRegressor(
                objective="reg:squarederror",
                random_state=CFG.RANDOM_SEED,
                n_jobs=-1,
                **model_params
            ))
        ]
    )

    scores = cross_val_score(
        pipeline,
        X_train_raw,
        y_train,
        cv=4,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )

    return -scores.mean()

In [8]:
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=CFG.RANDOM_SEED)
)

study.optimize(objective, n_trials=30)

[I 2026-01-16 15:14:52,239] A new study created in memory with name: no-name-56067fce-e2ce-4177-a182-52c9737de950
[I 2026-01-16 15:17:13,728] Trial 0 finished with value: 8.963016699712202 and parameters: {'n_estimators': 937, 'max_depth': 10, 'learning_rate': 0.08960785365368121, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'min_child_weight': 2, 'reg_alpha': 3.3323645788192616e-08, 'reg_lambda': 0.6245760287469893}. Best is trial 0 with value: 8.963016699712202.
[I 2026-01-16 15:19:54,054] Trial 1 finished with value: 8.771974928743035 and parameters: {'n_estimators': 1322, 'max_depth': 8, 'learning_rate': 0.010636066512540286, 'subsample': 0.9879639408647978, 'colsample_bytree': 0.9329770563201687, 'min_child_weight': 3, 'reg_alpha': 4.329370014459266e-07, 'reg_lambda': 4.4734294104626844e-07}. Best is trial 1 with value: 8.771974928743035.
[I 2026-01-16 15:21:15,940] Trial 2 finished with value: 8.759908012739505 and parameters: {'n_estimators': 817, 'ma

In [9]:
final_pipeline = Pipeline(
    steps=[
        ("preprocess", Preprocessor()),
        ("model", XGBRegressor(
            **study.best_params,
            objective="reg:squarederror",
            random_state=CFG.RANDOM_SEED,
            n_jobs=-1
        ))
    ]
)

final_pipeline.fit(X_train_raw, y_train)

y_pred = final_pipeline.predict(test)


In [10]:
# 最佳參數
print("Best RMSE:", study.best_value)
print("Best params:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

Best RMSE: 8.752383934649863
Best params:
n_estimators: 1625
max_depth: 7
learning_rate: 0.022868032935380388
subsample: 0.7716870468962868
colsample_bytree: 0.7581014414297597
min_child_weight: 4
reg_alpha: 0.00022496116000185
reg_lambda: 0.004939392464663163


In [11]:
# visualize data
#optuna.visualization.plot_optimization_history(study)
#optuna.visualization.plot_param_importances(study)

# Submission

In [12]:
submission = pd.read_csv(CFG.sample_submission_csv)
test_id = test["id"]
submission = pd.DataFrame({
    'Id': test_id, 
    'exam_score': y_pred   
})

submission.to_csv('submission.csv', index=False)

In [13]:
submission.head()

,Id,exam_score
0,630000,71.302765
1,630001,69.961716
2,630002,87.676155
3,630003,55.216450
4,630004,47.592976
